In [12]:
import pandas as pd

neg=pd.read_csv('../data/neg.csv',header=None,index_col=None)
pos=pd.read_csv('../data/pos.csv',header=None,index_col=None,error_bad_lines=False)
neu=pd.read_csv('../data/neutral.csv', header=None, index_col=None)

b'Skipping line 2607: expected 1 fields, saw 9\nSkipping line 3143: expected 1 fields, saw 2\nSkipping line 3173: expected 1 fields, saw 8\n'


In [13]:
neu[0]

0       做为一本声名在外的流行书，说的还是广州的外企，按道理应该和我的生存环境差不多啊。但是一看之下...
1       作者完全是以一个过来的自认为是成功者的角度去写这个问题，感觉很不客观。虽然不是很喜欢，但是，...
2            作者提倡内调，不信任化妆品，这点赞同。但是所列举的方法太麻烦，配料也不好找。不是太实用。
3       作者的文笔还行，但通篇感觉太琐碎，有点文人的无病呻吟。自由主义者。作者的品性不敢苟同，无民族...
4                           作者倒是个很小资的人,但有点自恋的感觉,书并没有什么大帮助
                              ...                        
4350                              虽然家里用不到 但商家的服务态度 超级棒！赞！
4351                  已经安装上了，但没有试，我相信美的质量，应该不会有问题。先5分好评吧。
4352    很不错，到货很快，当天给客服打电话，下午安装人员就上门了，速度啊!但但当时没有水，没有试试，...
4353    买来放在出租房里的，所以自己也没试过，但是安装服务人员特别好，最大限度地给省钱，两套热水澡装...
4354    买来放在出租房里的，所以自己也没试过，但是安装服务人员特别好，最大限度地给省钱，两套热水澡装...
Name: 0, Length: 4355, dtype: object

In [14]:
import numpy as np

combined = np.concatenate((pos[0], neu[0], neg[0]))
combined.shape

(21088,)

In [15]:
# pos -> 1; neu -> 0; neg -> -1
y = np.concatenate((np.ones(len(pos), dtype=int), np.zeros(len(neu), dtype=int), -1*np.ones(len(neg),dtype=int)))
y.shape

(21088,)

In [16]:
import jieba

#对句子经行分词，并去掉换行符
def tokenizer(text):
    ''' Simple Parser converting each document to lower-case, then
        removing the breaks for new lines and finally splitting on the
        whitespace
    '''
    text = [jieba.lcut(document.replace('\n', '')) for document in text]
    return text

combined = tokenizer(combined)

In [19]:
combined

[['做',
  '父母',
  '一定',
  '要',
  '有',
  '刘墉',
  '这样',
  '的',
  '心态',
  '，',
  '不断',
  '地',
  '学习',
  '，',
  '不断',
  '地',
  '进步',
  '，',
  '不断',
  '地',
  '给',
  '自己',
  '补充',
  '新鲜血液',
  '，',
  '让',
  '自己',
  '保持',
  '一颗',
  '年轻',
  '的',
  '心',
  '。',
  '我',
  '想',
  '，',
  '这',
  '是',
  '他',
  '能',
  '很',
  '好',
  '的',
  '和',
  '孩子',
  '沟通',
  '的',
  '一个',
  '重要',
  '因素',
  '。',
  '读',
  '刘墉',
  '的',
  '文章',
  '，',
  '总能',
  '让',
  '我',
  '看到',
  '一个',
  '快乐',
  '的',
  '平易近人',
  '的',
  '父亲',
  '，',
  '他',
  '始终',
  '站',
  '在',
  '和',
  '孩子',
  '同样',
  '的',
  '高度',
  '，',
  '给',
  '孩子',
  '创造',
  '着',
  '一个',
  '充满',
  '爱',
  '和',
  '自由',
  '的',
  '生活',
  '环境',
  '。',
  '很',
  '喜欢',
  '刘墉',
  '在',
  '字里行间',
  '流露出',
  '的',
  '做',
  '父母',
  '的',
  '那种',
  '小',
  '狡黠',
  '，',
  '让',
  '人',
  '总是',
  '忍俊不禁',
  '，',
  '父母',
  '和',
  '子女',
  '之间',
  '有时候',
  '也',
  '是',
  '一种',
  '战斗',
  '，',
  '武力',
  '争斗',
  '过于',
  '低级',
  '了',
  '，',
  '智力',
  '较量',
  '才',
  '更',
  '有',
  '趣味',
  '。',
  '

In [32]:
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from keras.preprocessing import sequence
import multiprocessing

cpu_count = multiprocessing.cpu_count() # 4
vocab_dim = 100
n_iterations = 10  # ideally more..
n_exposures = 10 # 所有频数超过10的词语
window_size = 7
n_epoch = 4
input_length = 100
maxlen = 100

def create_dictionaries(model=None,
                        combined=None):
    ''' Function does are number of Jobs:
        1- Creates a word to index mapping
        2- Creates a word to vector mapping
        3- Transforms the Training and Testing Dictionaries

    '''
    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.vocab.keys(),
                            allow_update=True)
        #  freqxiao10->0 所以k+1
        w2indx = {v: k+1 for k, v in gensim_dict.items()}#所有频数超过10的词语的索引,(k->v)=>(v->k)
        w2vec = {word: model[word] for word in w2indx.keys()}#所有频数超过10的词语的词向量, (word->model(word))

        def parse_dataset(combined): # 闭包-->临时使用
            ''' Words become integers
            '''
            data=[]
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0) # freqxiao10->0
                data.append(new_txt)
            return data # word=>index
        combined=parse_dataset(combined)
        combined= sequence.pad_sequences(combined, maxlen=maxlen)#每个句子所含词语对应的索引，所以句子中含有频数小于10的词语，索引为0
        return w2indx, w2vec,combined
    else:
        print ('No data provided...')


#创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引
def word2vec_train(combined):

    model = Word2Vec(size=vocab_dim,
                     min_count=n_exposures,
                     window=window_size,
                     workers=cpu_count,
                     iter=n_iterations)
    model.build_vocab(combined) # input: list
    model.train(combined,epochs=4,total_examples=21088)
    model.save('../model/Word2vec_model.pkl')
    print(model)
    index_dict, word_vectors,combined = create_dictionaries(model=model,combined=combined)
    return   index_dict, word_vectors,combined

print( 'Training a Word2vec model...')
index_dict, word_vectors,combined=word2vec_train(combined)

Training a Word2vec model...
Word2Vec(vocab=8308, size=100, alpha=0.025)


AttributeError: 'Word2Vec' object has no attribute 'vocab'

NameError: name 'Word2Vec' is not defined

In [18]:
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout,Activation
from keras.models import model_from_yaml
np.random.seed(1337)  # For Reproducibility
import sys
sys.setrecursionlimit(1000000)
import yaml
import keras

batch_size = 32


def get_data(index_dict,word_vectors,combined,y):

    n_symbols = len(index_dict) + 1  # 所有单词的索引数，频数小于10的词语索引为0，所以加1
    embedding_weights = np.zeros((n_symbols, vocab_dim)) # 初始化 索引为0的词语，词向量全为0
    for word, index in index_dict.items(): # 从索引为1的词语开始，对每个词语对应其词向量
        embedding_weights[index, :] = word_vectors[word]
    x_train, x_test, y_train, y_test = train_test_split(combined, y, test_size=0.2)
    y_train = keras.utils.to_categorical(y_train,num_classes=3) 
    y_test = keras.utils.to_categorical(y_test,num_classes=3)
    # print x_train.shape,y_train.shape
    return n_symbols,embedding_weights,x_train,y_train,x_test,y_test


##定义网络结构
def train_lstm(n_symbols,embedding_weights,x_train,y_train,x_test,y_test):
    print 'Defining a Simple Keras Model...'
    model = Sequential()  # or Graph or whatever
    model.add(Embedding(output_dim=vocab_dim,
                        input_dim=n_symbols,
                        mask_zero=True,
                        weights=[embedding_weights],
                        input_length=input_length))  # Adding Input Length
    model.add(LSTM(output_dim=50, activation='tanh', inner_activation='hard_sigmoid'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax')) # Dense=>全连接层,输出维度=1
    model.add(Activation('softmax'))

    print 'Compiling the Model...'
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',metrics=['accuracy'])

    print "Train..." # batch_size=32
    model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epoch,verbose=1)

    print "Evaluate..."
    score = model.evaluate(x_test, y_test,
                                batch_size=batch_size)

    yaml_string = model.to_yaml()
    with open('../model/lstm.yml', 'w') as outfile:
        outfile.write( yaml.dump(yaml_string, default_flow_style=True) )
    model.save_weights('../model/lstm.h5')
    print 'Test score:', score

print 'Setting up Arrays for Keras Embedding Layer...'
n_symbols,embedding_weights,x_train,y_train,x_test,y_test=get_data(index_dict, word_vectors,combined,y)
print "x_train.shape and y_train.shape:"
print x_train.shape,y_train.shape
train_lstm(n_symbols,embedding_weights,x_train,y_train,x_test,y_test)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('Defining a Simple Keras Model...')? (<ipython-input-18-97b1fe08d348>, line 31)

In [11]:
"""
预测
"""
import jieba
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from keras.preprocessing import sequence

import yaml
from keras.models import model_from_yaml
np.random.seed(1337)  # For Reproducibility
import sys
sys.setrecursionlimit(1000000)

# define parameters
maxlen = 100

def create_dictionaries(model=None,
                        combined=None):
    ''' Function does are number of Jobs:
        1- Creates a word to index mapping
        2- Creates a word to vector mapping
        3- Transforms the Training and Testing Dictionaries

    '''
    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.vocab.keys(),
                            allow_update=True)
        #  freqxiao10->0 所以k+1
        w2indx = {v: k+1 for k, v in gensim_dict.items()}#所有频数超过10的词语的索引,(k->v)=>(v->k)
        w2vec = {word: model[word] for word in w2indx.keys()}#所有频数超过10的词语的词向量, (word->model(word))

        def parse_dataset(combined): # 闭包-->临时使用
            ''' Words become integers
            '''
            data=[]
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0) # freqxiao10->0
                data.append(new_txt)
            return data # word=>index
        combined=parse_dataset(combined)
        combined= sequence.pad_sequences(combined, maxlen=maxlen)#每个句子所含词语对应的索引，所以句子中含有频数小于10的词语，索引为0
        return w2indx, w2vec,combined
    else:
        print 'No data provided...'


def input_transform(string):
    words=jieba.lcut(string)
    words=np.array(words).reshape(1,-1)
    model=Word2Vec.load('../model/Word2vec_model.pkl')
    _,_,combined=create_dictionaries(model,words)
    return combined


def lstm_predict(string):
    print 'loading model......'
    with open('../model/lstm.yml', 'r') as f:
        yaml_string = yaml.load(f)
    model = model_from_yaml(yaml_string)

    print 'loading weights......'
    model.load_weights('../model/lstm.h5')
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',metrics=['accuracy'])
    data=input_transform(string)
    data.reshape(1,-1)
    #print data
    result=model.predict_classes(data)
    print result # [[1]]
    if result[0]==1:
        print string,' positive'
    elif result[0]==0:
        print string,' neural'
    else:
        print string,' negative'

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('No data provided...')? (<ipython-input-11-a173c2961a22>, line 52)

In [10]:
# string='酒店的环境非常好，价格也便宜，值得推荐'
# string='手机质量太差了，傻逼店家，赚黑心钱，以后再也不会买了'
# string = "这是我看过文字写得很糟糕的书，因为买了，还是耐着性子看完了，但是总体来说不好，文字、内容、结构都不好"
# string = "虽说是职场指导书，但是写的有点干涩，我读一半就看不下去了！"
# string = "书的质量还好，但是内容实在没意思。本以为会侧重心理方面的分析，但实际上是婚外恋内容。"
string = "不是太好"
# string = "不错不错"
# string = "非常好非常好！！"
# string = "真的一般，没什么可以学习的"

lstm_predict(string)

NameError: name 'lstm_predict' is not defined